In [ ]:
import pickle
import numpy as np
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from pprint import pprint
from time import time
import logging



#nltk.download('stopwords')

In [ ]:
with open('NHlegtext.corpus','rb') as f:
    legcorpus = pickle.load(f)
#remove legslative stop words (if not accomplished in the legislation retrieval)
#leg_stop_words = ['rep','sen','bold','italics','sponsors']

print(legcorpus[303]) #301, 302

In [ ]:
#make a dataset on just the bill

#combine each document into a single document
combined_docs = {} #{bill_id:text}
total = 0
amend_count = 0
passed = 0
for doc in legcorpus:
    total+=1
    pass_flag = 0
    if doc[0] not in combined_docs:
        try:
            if doc[2][-5] == 'approved':
                doc[2] = doc[2][:-5]
                pass_flag = 1
                passed += 1
        except:
            pass
        combined_docs[doc[0]] = " ".join(doc[2])
    else:
        combined_docs[doc[0]] += " ".join(doc[2])
        amend_count+=1
        #if pass_flag == 1:
        #    passed
        #print(" ".join(doc[2][-20:]))
        
#print(combined_docs["1035888"])# 1035888 passed

#bill_texts = [combined_docs[i] for i in combined_docs.keys()]
#print(bill_texts[0])
print(total)
print(amend_count)
print(amend_count/total)

In [ ]:
print(combined_docs['909012'])#"1035888"])'894704'

In [ ]:
leg_data = pd.read_csv("leg_data.csv")
leg_target = pd.read_csv("leg_win_state.csv")
print(type(leg_data["bill_id"][1]))
leg_data.head()

In [ ]:
bill_order = []
bill_target = []
for i, bill_id in enumerate(leg_data["bill_id"]):
    if str(bill_id) not in bill_order:
        bill_order.append(str(bill_id))
        bill_target.append(leg_target["every stage passed"][i])

In [ ]:
bill_texts = []
for bill_id in bill_order:
    if bill_id in combined_docs:
        bill_texts.append(combined_docs[bill_id])
    else:
        bill_texts.append("") #we don't have the document, so append an empty document


In [ ]:
print(len(bill_order))
print(len(bill_target))
print(len(bill_texts))

#print(bill_order[100])
#print(bill_target[100])
#print(bill_texts[100])

In [ ]:
amount_train = 1830
amount_test = 300
X_train = bill_texts[0:amount_train]
y_train = bill_target[0:amount_train]
X_test = bill_texts[amount_train:amount_train+amount_test]
y_test = bill_target[amount_train:amount_train+amount_test]

print(sum(y_train)/len(y_train))
print(sum(y_test)/len(y_test))

In [ ]:
#https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py

# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_accuracy'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation accuracy: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_accuracy'][candidate],
                  results['std_test_accuracy'][candidate]))
            print('Precision: ', results['mean_test_precision_micro'][candidate],
                  'Recall: ', results['mean_test_recall_micro'][candidate],
                  'f1: ', results['mean_test_f1_micro'][candidate])
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
#from https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
stemmer = SnowballStemmer("english", ignore_stopwords=True)
#adjust this stemmer so that it does more than one word in the tokens


class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])


In [ ]:
stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(1,4))#, max_df=0.75)# min_df=1

text_clf = Pipeline([
    ('vect', stemmed_count_vect),
    ('tfidf', TfidfTransformer()),
    ('feature_select', SelectKBest(f_classif, k=1400)),# chi2, f_classif, mutual_info_classif
    ('clf', MultinomialNB(#MLPClassifier(#(#SGDClassifier(loss='hinge', penalty='l2', #),#  # #
                          alpha=0.0#1e-4,#1e-3,
                          #random_state=42,
                          #max_iter=250
                         )),
])

text_clf.fit(X_train, y_train)
predicted = text_clf.predict(X_test)
#predicted_prob = text_clf.predict_proba(X_test) #doesnt work with 'hinge' loss function
np.mean(predicted == y_test)

In [ ]:
feature_indices = text_clf.__dict__['steps'][2][1].get_support(indices=True)
#fail_probs = text_clf.__dict__['steps'][3][1].feature_log_prob_[0] #__dict__#.coef_[0]#
pass_probs = text_clf.__dict__['steps'][3][1].feature_log_prob_[1]
#print(len(pass_probs))
#print(len(feature_indices))
feature_pass_probs = {feature:pass_probs[i] for i, feature in enumerate(feature_indices)}

sorted_features = sorted(feature_indices, key=lambda x: feature_pass_probs[x], reverse=True)
#print(sorted_features)

vocab_dict = text_clf.__dict__['steps'][0][1].__dict__['vocabulary_']

#invert the vocab_dict
index_vocab = {}
for key in vocab_dict.keys():
    index_vocab[int(vocab_dict[key])] = key #indices for keys, words for values
    
for i, feature in enumerate(sorted_features):
    if i >= 1000:
        break
    print(index_vocab[feature], "\t\t", feature_pass_probs[feature])

In [ ]:
vocab_dict = text_clf.__dict__['steps'][0][1].__dict__['vocabulary_']
print(vocab_dict)

The below cell prints out the features chosen by this classifier. 
* We may need to weed out phrases that include the words "approved" or "effective date", which indicate that these have already passed. That, or only look at the first docement associated with a bill (the thing that actually has the law in it, and not the notes on where the bill is moving).
* Or, we can select the bills that dont have (too many) top-200 words, and train on those

In [ ]:
#print the vocabulary
# for i, key in enumerate(vocab_dict):
#     print(key, ", ", vocab_dict[key])
#     if i > 100:
#         break

vocab_dict = text_clf.__dict__['steps'][0][1].__dict__['vocabulary_']

#invert the vocab_dict
index_vocab = {}
for key in vocab_dict.keys():
    index_vocab[int(vocab_dict[key])] = key #indices for keys, words for values
    
    
feature_indices = text_clf.__dict__['steps'][2][1].get_support(indices=True)
print(feature_indices)

for i in feature_indices:
    #print(i, "\t\t", index_vocab[i], text_clf.__dict__['steps'][3][1].__dict__['feature_log_prob_'])
    print(index_vocab[i])#, text_clf.__dict__['steps'][3][1].__dict__['feature_log_prob_'])
    
 

In [ ]:
#dense_row_zero = 
#print(len(dense_row_zero[0]))
#for i in range(len(dense_row_zero[0])):
#    num = dense_row_zero[0][i]
#    if num != 0:
#        print(i, num)

In [ ]:
vect_train = stemmed_count_vect(X_train)
tfidf_train = TfidfTransformer(vect_train)

vect_test = stemmed_count_vect(X_test)
tfidf_test = TfidfTransformer(vect_test)



text_clf_2 = Pipeline([
    ('feature_select', SelectKBest(f_classif, k=250)),# chi2, f_classif, mutual_info_classif
    ('clf', MLPClassifier(#SGDClassifier(loss='hinge', penalty='l2', #MultinomialNB()),#  #
                          alpha=0.0001,#1e-3,
                          random_state=42,
                          max_iter=200)),
])

text_clf_2.fit(tfidf_train, y_train)
predicted = text_clf_2.predict(stemmed_count_vect(X_test))
#predicted_prob = text_clf.predict_proba(X_test) #doesnt work with 'hinge' loss function
np.mean(predicted == y_test)

In [ ]:
parameters = {
    #'vect__max_df': (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1,4), (1,5)),  # unigrams, bigrams, or trigrams
    # 'tfidf__use_idf': (True, False),
    # 'tfidf__norm': ('l1', 'l2'),
    #'feature_select__score_func': (chi2, f_classif),
    'feature_select__k': (128, 160, 180, 200, 225, 256), #possibly the most important parameter
    'clf__max_iter': (128, 192, 220, 256),
    'clf__alpha': (0.0001, 0.00001, 0.000001)
    
}

if __name__ == "__main__":
    
    #grid_search = GridSearchCV(text_clf, parameters, n_jobs=1, verbose=1)
    
    n_iter_search = 160
    random_search = RandomizedSearchCV(text_clf, param_distributions=parameters,
                                   n_iter=n_iter_search, cv=3, iid=False, refit='accuracy')

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in text_clf.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    random_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % random_search.best_score_)
    print("Best parameters set:")
    best_parameters = random_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    #report(random_search.cv_results_)
    
 

In [ ]:
X_train.shape

In [ ]:
print(np.mean(predicted == y_test))
#predicted_prob = text_clf.predict_proba(X_test)
#print(predicted_prob)

In [ ]:
print(stemmed_count_vect)

In [ ]:
#Added by Ross
# playing around with topic models
# https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py
def print_top_words(model, feature_names, n_top_words):
    total_message = []
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        
        temp = " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        total_message.append(temp)
        print(message)
    print()
    return total_message

n_features = 2000
n_top_words = 1
n_components = 40 #number of topics


data_samples = bill_texts[0:amount_train]
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=n_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data_samples)
tf_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=n_features,stop_words='english')
tf = stemmed_count_vect.fit_transform(data_samples)
#print(tf)
nmf = NMF(n_components=n_components, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

print_top_words(nmf, tfidf_feature_names, n_top_words)
lda = LatentDirichletAllocation(n_components=n_components, max_iter=10,
                                learning_method='online', learning_offset=50.,random_state=4)

lda.fit(tf)

print("\nTopics in LDA model:")
tf_feature_names = tfidf_feature_names.get_feature_names()
message = print_top_words(lda, tf_feature_names, n_top_words)
print(message)
print(lda.components_)

